In [ ]:
import cudf
import pyarrow as pa
import pandas as pd
import numpy as np
import cupy as cp

In [ ]:
df = cudf.read_parquet('./all_states_us_lat_long.parquet/*').reset_index().drop(['index'])

In [ ]:
# df['block_id'] = (df.GISJOIN%10000).astype('int32')
# df['State'] = (df.GISJOIN/1000_000_000_000_000).astype('int32')
df['GISJOIN_og'] = df.GISJOIN
df['GISJOIN'] = (df.GISJOIN/1000).astype('int')

In [ ]:
df

## Adding gender block wise

In [ ]:
df_cat = cudf.read_csv('/home/ajay/data/census/all_states_block_group/nhgis0016_csv/nhgis0016_ds176_20105_2010_blck_grp.csv')

df_cat.STATEA = df_cat.STATEA.astype('int')
df_cat = df_cat.query('STATEA != 72')

df_cat.GISJOIN = df_cat.GISJOIN.str.replace('G', '').astype('int')

# sex_by_age_cols = ['JLZE001','JLZE002','JLZE003','JLZE004','JLZE005','JLZE006','JLZE007','JLZE008','JLZE009','JLZE010','JLZE011','JLZE012','JLZE013','JLZE014','JLZE015','JLZE016','JLZE017','JLZE018','JLZE019','JLZE020','JLZE021','JLZE022','JLZE023','JLZE024','JLZE025','JLZE026','JLZE027','JLZE028','JLZE029','JLZE030','JLZE031','JLZE032','JLZE033','JLZE034','JLZE035','JLZE036','JLZE037','JLZE038','JLZE039','JLZE040','JLZE041','JLZE042','JLZE043','JLZE044','JLZE045','JLZE046','JLZE047','JLZE048','JLZE049']
sex_by_age_cols = ['JLZE001', 'JLZE002', 'JLZE026']
df_cat = df_cat[['GISJOIN']+sex_by_age_cols]

for i in df_cat.columns:
    if i != 'GISJOIN':
        df_cat[i] = df_cat[i].astype('int32')

In [ ]:
df_cat.JLZE001.value_counts()

In [ ]:
df_cat.JLZE001.value_counts()

In [ ]:
df_cat.JLZE002 = df_cat.JLZE002/df_cat.JLZE001
df_cat.JLZE026 = df_cat.JLZE026/df_cat.JLZE001

In [ ]:
df_cat = df_cat.nans_to_nulls()

In [ ]:
df_cat = df_cat.fillna(0)

In [ ]:
df_cat.JLZE002 = df_cat.JLZE002.fillna(0)
df_cat.JLZE026 = df_cat.JLZE026.fillna(0)

In [ ]:
df_cat.JLZE026.unique()

In [ ]:
df_cat.JLZE002.unique()

In [ ]:
df_cat.drop_column('JLZE001')
df_cat

In [ ]:
temp = df.merge(df_cat, on='GISJOIN')

In [ ]:
del(df)
del(df_cat)

In [ ]:
temp.drop_column('GISJOIN')

In [ ]:
# temp.index = temp.GISJOIN_og
temp.rename({'GISJOIN_og':'GISJOIN'}, inplace=True)
temp

In [ ]:
temp.to_parquet('temp.parquet')